In [1]:
# Dans votre notebook
import pandas as pd

# Charger les features
df = pd.read_parquet('../data/features_engineered.parquet')

# Séparer train et test
train = df[df['TARGET'].notna()].copy()  # Lignes avec TARGET → train
test = df[df['TARGET'].isna()].copy()    # Lignes sans TARGET → test Kaggle

# Préparer pour l'entraînement
X_train = train.drop(['TARGET', 'SK_ID_CURR'], axis=1)
y_train = train['TARGET']

# Pour les prédictions finales
X_test = test.drop(['TARGET', 'SK_ID_CURR'], axis=1)
test_ids = test['SK_ID_CURR']

print(f"Train : {X_train.shape}")
print(f"Test : {X_test.shape}")

Train : (307507, 795)
Test : (48744, 795)


In [2]:
import mlflow
import os

# Configuration MLflow en local (pas Docker)
# Les runs et artifacts seront stockés dans ./mlruns à la racine du projet
tracking_uri = os.path.abspath(os.path.join(os.getcwd(), '..', 'mlruns'))
mlflow.set_tracking_uri(f"file://{tracking_uri}")
mlflow.set_experiment("Projet MLOps - Credit Scoring - Random Forest")

print(f"Tracking URI: {mlflow.get_tracking_uri()}")
print(f"Répertoire de stockage: {tracking_uri}")
print(f"✅ MLflow configuré en local")

# Vérifier la connexion
try:
    experiments = mlflow.search_experiments()
    print(f"Nombre d'expériences: {len(experiments)}")
except Exception as e:
    print(f"⚠️ Erreur: {e}")


Tracking URI: file:///home/zmxw1768/Documents/oc_mlops/mlruns
Répertoire de stockage: /home/zmxw1768/Documents/oc_mlops/mlruns
✅ MLflow configuré en local
Nombre d'expériences: 2


In [3]:

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
# Séparer avec StratifiedKFold pour garder la même proportion de classes
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.2, stratify=y_train, random_state=42)

In [4]:
# Vérifier et nettoyer les valeurs problématiques sur TOUS les datasets
import numpy as np

print("🧹 Nettoyage des données...")

# Nettoyer X_train (utilisé pour la CV)
X_train = X_train.replace([np.inf, -np.inf], np.nan).fillna(0)

# Nettoyer X_tr et X_val (utilisés pour les tests simples)
X_tr = X_tr.replace([np.inf, -np.inf], np.nan).fillna(0)
X_val = X_val.replace([np.inf, -np.inf], np.nan).fillna(0)

# Nettoyer X_test (pour les prédictions finales)
X_test = X_test.replace([np.inf, -np.inf], np.nan).fillna(0)

print(f"✅ Données nettoyées")
print(f"   X_train: {X_train.shape}")
print(f"   X_test: {X_test.shape}")


🧹 Nettoyage des données...
✅ Données nettoyées
   X_train: (307507, 795)
   X_test: (48744, 795)


In [5]:
# Vérifier la distribution des classes
print("📊 Distribution des classes:")
print("-" * 50)
print(f"Classe 0 (Pas de défaut): {(y_train == 0).sum():,} ({(y_train == 0).mean()*100:.2f}%)")
print(f"Classe 1 (Défaut):        {(y_train == 1).sum():,} ({(y_train == 1).mean()*100:.2f}%)")
print(f"Ratio déséquilibre: 1:{(y_train == 0).sum() / (y_train == 1).sum():.1f}")
print("-" * 50)
print(f"Total: {len(y_train):,} échantillons")


📊 Distribution des classes:
--------------------------------------------------
Classe 0 (Pas de défaut): 282,682 (91.93%)
Classe 1 (Défaut):        24,825 (8.07%)
Ratio déséquilibre: 1:11.4
--------------------------------------------------
Total: 307,507 échantillons


In [6]:
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (roc_auc_score, recall_score, f1_score, 
                             make_scorer, confusion_matrix)
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import numpy as np

# Fermer toute run active
if mlflow.active_run():
    mlflow.end_run()

# Définir le coût métier personnalisé (FN coûte 10x plus que FP)
def business_cost_scorer(y_true, y_pred):
    """
    Coût métier : FN (faux négatif) coûte 10 fois plus cher que FP (faux positif)
    On retourne le négatif du coût pour maximiser (sklearn maximise les scores)
    """
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    cost = fp * 1 + fn * 10  # FN coûte 10x plus
    return -cost  # Négatif car on veut minimiser le coût

# Configuration de la validation croisée
n_splits = 3  # Réduit à 3 pour éviter la surcharge CPU
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Définir les modèles avec class_weight='balanced' pour gérer le déséquilibre
models = {
    "Logistic Regression": Pipeline([
        ('scaler', StandardScaler()),  # ← NORMALISATION OBLIGATOIRE
        ('classifier', LogisticRegression(
            max_iter=1000,  # Suffisant avec normalisation
            random_state=42,
            class_weight='balanced',
            solver='lbfgs',
            verbose=1
            # n_jobs retiré (déprécié)
        ))
    ]),
    
    "Random Forest": RandomForestClassifier(
        n_estimators=50, 
        random_state=42, 
        n_jobs=-1,  # ← OK pour Random Forest
        max_depth=10,
        class_weight='balanced',
        verbose=1
    )
}

# Définir les scorers (response_method au lieu de needs_proba pour sklearn récent)
scoring = {
    'roc_auc': make_scorer(roc_auc_score, response_method='predict_proba'),
    'recall_minority': make_scorer(recall_score, pos_label=1, zero_division=0),
    'f1': make_scorer(f1_score, pos_label=1, zero_division=0),
    'business_cost': make_scorer(business_cost_scorer)
}

print(f"🔄 Validation croisée avec {n_splits} folds")
print(f"📊 Métriques : ROC-AUC, Recall (classe 1), F1-Score, Coût métier (FN=10x FP)")
print(f"⚙️  Configuration : class_weight='balanced' pour gérer le déséquilibre\n")

# Tester chaque modèle
results = {}
for model_name, model in models.items():
    print(f"🤖 Entraînement : {model_name}")
    
    with mlflow.start_run(run_name=f"{model_name} - CV {n_splits} folds - Balanced"):
        # 🏷️ TAGS ET METADATA
        mlflow.set_tags({
            "project": "Home Credit Default Risk",
            "phase": "baseline",
            "model_type": "classification",
            "data_version": "v1.0",
            "environment": "development"
        })
        
        # 📝 DESCRIPTION/NOTE (apparaît dans l'interface MLflow)
        mlflow.set_tag("mlflow.note.content", 
            f"""Entraînement {model_name} avec validation croisée stratifiée.
            
            Objectif: Détecter les défauts de paiement avec gestion du déséquilibre de classes.
            Stratégie: class_weight='balanced' + StandardScaler.
            Métrique prioritaire: Coût métier (FN=10x FP).
            
            Date: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M')}
            """)
        
        # Validation croisée avec plusieurs métriques
        cv_results = cross_validate(
            model, 
            X_train, 
            y_train, 
            cv=skf, 
            scoring=scoring,
            n_jobs=1,
            return_train_score=False,
            error_score='raise'  # Lever les erreurs pour debugging
        )
        
        # Logger les paramètres
        mlflow.log_param("model", model_name)
        mlflow.log_param("cv_strategy", "StratifiedKFold")
        mlflow.log_param("n_splits", n_splits)
        mlflow.log_param("n_samples", len(X_train))
        mlflow.log_param("n_features", X_train.shape[1])
        mlflow.log_param("class_weight", "balanced")
        mlflow.log_param("business_cost_ratio", "FN=10x FP")
        
        # Logger les paramètres spécifiques du modèle
        for param_name, param_value in model.get_params().items():
            mlflow.log_param(f"model_{param_name}", str(param_value))
        
        # Stocker et logger les résultats pour chaque métrique
        results[model_name] = {}
        
        for metric_name in scoring.keys():
            scores = cv_results[f'test_{metric_name}']
            mean_score = np.nanmean(scores)  # Utiliser nanmean pour gérer les NaN
            std_score = np.nanstd(scores)
            
            # Stocker les résultats
            results[model_name][metric_name] = {
                'mean': mean_score,
                'std': std_score,
                'scores': scores
            }
            
            # Logger dans MLflow
            mlflow.log_metric(f"{metric_name}_mean", mean_score)
            mlflow.log_metric(f"{metric_name}_std", std_score)
            
            # Logger les scores de chaque fold
            for i, score in enumerate(scores, 1):
                mlflow.log_metric(f"{metric_name}_fold_{i}", score)
            
            # Affichage formaté
            metric_display = metric_name.replace('_', ' ').title()
            print(f"   {metric_display:20s}: {mean_score:.4f} (±{std_score:.4f})")
        
        # Entraîner le modèle sur toutes les données pour le sauvegarder
        print("   📦 Sauvegarde du modèle...")
        model.fit(X_train, y_train)
        
        # Diagnostics sur le modèle entraîné
        y_pred_train = model.predict(X_train[:1000])
        print(f"   🔍 Prédictions sur 1000 échantillons: {np.sum(y_pred_train)} positifs")
        
        # Créer la signature et l'exemple
        if hasattr(model, 'predict_proba'):
            y_pred_proba = model.predict_proba(X_train[:100])[:, 1]
            signature = mlflow.models.signature.infer_signature(X_train, y_pred_proba)
        else:
            signature = mlflow.models.signature.infer_signature(X_train, model.predict(X_train))
        
        input_example = X_train.head(3)
        
        # Logger le modèle
        mlflow.sklearn.log_model(
            model,
            "model",
            signature=signature,
            input_example=input_example
        )
        
        print()

print("\n" + "="*80)
print("📈 RÉSUMÉ DES PERFORMANCES")
print("="*80)
for model_name, metrics in results.items():
    print(f"\n{model_name}:")
    print("-" * 80)
    print(f"{'Métrique':<25s} {'Mean':>12s} {'Std':>12s}")
    print("-" * 80)
    for metric_name, result in metrics.items():
        metric_display = metric_name.replace('_', ' ').title()
        print(f"{metric_display:<25s} {result['mean']:>12.4f} {result['std']:>12.4f}")

print("\n" + "="*80)
# Utiliser nanmax pour gérer les NaN
print(f"✅ Meilleur AUC-ROC: {max(results, key=lambda k: results[k]['roc_auc']['mean'] if not np.isnan(results[k]['roc_auc']['mean']) else -1)}")
print(f"✅ Meilleur Recall: {max(results, key=lambda k: results[k]['recall_minority']['mean'])}")
print(f"✅ Meilleur F1-Score: {max(results, key=lambda k: results[k]['f1']['mean'])}")
print(f"✅ Meilleur Coût Métier: {max(results, key=lambda k: results[k]['business_cost']['mean'])}")
print("="*80)


🔄 Validation croisée avec 3 folds
📊 Métriques : ROC-AUC, Recall (classe 1), F1-Score, Coût métier (FN=10x FP)
⚙️  Configuration : class_weight='balanced' pour gérer le déséquilibre

🤖 Entraînement : Logistic Regression
   Roc Auc             : 0.7670 (±0.0021)
   Recall Minority     : 0.6983 (±0.0062)
   F1                  : 0.2753 (±0.0014)
   Business Cost       : -52892.3333 (±404.6334)
   📦 Sauvegarde du modèle...
   🔍 Prédictions sur 1000 échantillons: 340.0 positifs


2026/02/10 13:36:05 WARNING mlflow.models.signature: Failed to infer schema for inputs. Setting schema to `Schema([ColSpec(type=AnyType())]` as default. Note that MLflow doesn't validate data types during inference for AnyType. To see the full traceback, set logging level to DEBUG.
2026/02/10 13:36:08 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2026/02/10 13:36:08 WARNING mlflow.models.model: Failed to validate serving input example {
  "dataframe_split": {
    "columns": [
      "CODE_GENDER",
      "FLAG_OWN_CAR",
      "FLAG_OWN_REALTY",
      "CNT_CHILDREN",
      "AMT_INCOME_TOTAL",
      "AMT_CREDIT",
      "AMT_ANNUITY",
      "AMT_GOODS_PRICE",
      "REGION_POPULATION_RELATIVE",
      "DAYS_BIRTH",
      "DAYS_EMPLOYED",
      "DAYS_REGISTRATION",
      "DAYS_ID_PUBLISH",
      "OWN_CAR_AGE",
      "FLAG_MOBIL",
      "FLAG_EMP_PHONE",
      "FLAG_WORK_PHONE",
     


🤖 Entraînement : Random Forest


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    7.0s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   10.5s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.1s
[Parallel(n_jobs=12)]: Done  50 out of  50 | elapsed:    0.1s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.1s
[Parallel(n_jobs=12)]: Done  50 out of  50 | elapsed:    0.2s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    7.0s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   10.4s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0

   Roc Auc             : 0.7349 (±0.0018)
   Recall Minority     : 0.5214 (±0.0030)
   F1                  : 0.2729 (±0.0026)
   Business Cost       : -58635.6667 (±339.0028)
   📦 Sauvegarde du modèle...


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   11.5s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   17.6s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done  50 out of  50 | elapsed:    0.0s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done  50 out of  50 | elapsed:    0.0s finished


   🔍 Prédictions sur 1000 échantillons: 241.0 positifs


2026/02/10 13:39:05 WARNING mlflow.models.signature: Failed to infer schema for inputs. Setting schema to `Schema([ColSpec(type=AnyType())]` as default. Note that MLflow doesn't validate data types during inference for AnyType. To see the full traceback, set logging level to DEBUG.
2026/02/10 13:39:08 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2026/02/10 13:39:08 WARNING mlflow.models.model: Failed to validate serving input example {
  "dataframe_split": {
    "columns": [
      "CODE_GENDER",
      "FLAG_OWN_CAR",
      "FLAG_OWN_REALTY",
      "CNT_CHILDREN",
      "AMT_INCOME_TOTAL",
      "AMT_CREDIT",
      "AMT_ANNUITY",
      "AMT_GOODS_PRICE",
      "REGION_POPULATION_RELATIVE",
      "DAYS_BIRTH",
      "DAYS_EMPLOYED",
      "DAYS_REGISTRATION",
      "DAYS_ID_PUBLISH",
      "OWN_CAR_AGE",
      "FLAG_MOBIL",
      "FLAG_EMP_PHONE",
      "FLAG_WORK_PHONE",
     



📈 RÉSUMÉ DES PERFORMANCES

Logistic Regression:
--------------------------------------------------------------------------------
Métrique                          Mean          Std
--------------------------------------------------------------------------------
Roc Auc                         0.7670       0.0021
Recall Minority                 0.6983       0.0062
F1                              0.2753       0.0014
Business Cost              -52892.3333     404.6334

Random Forest:
--------------------------------------------------------------------------------
Métrique                          Mean          Std
--------------------------------------------------------------------------------
Roc Auc                         0.7349       0.0018
Recall Minority                 0.5214       0.0030
F1                              0.2729       0.0026
Business Cost              -58635.6667     339.0028

✅ Meilleur AUC-ROC: Logistic Regression
✅ Meilleur Recall: Logistic Regression
✅ Meilleur

## 🏷️ Tags et Organisation MLflow

### Tags recommandés par catégorie :

**Métadonnées projet :**
- `author` : Auteur de l'expérience
- `project` : Nom du projet
- `version` : Version du code/données
- `environment` : dev / staging / production

**Contexte expérience :**
- `phase` : exploration / baseline / optimization / production
- `model_type` : classification / regression / clustering
- `approach` : feature_engineering / hyperparameter_tuning / ensemble

**Tags techniques :**
- `gpu_used` : True/False
- `framework` : sklearn / xgboost / pytorch
- `data_sampling` : full / stratified / undersampling

**Tags métier :**
- `business_priority` : cost_reduction / accuracy / speed
- `deployment_ready` : True/False
- `validation_status` : pending / approved / rejected

### Tag spécial : Description

Le tag **`mlflow.note.content`** apparaît dans l'interface UI comme une note éditable.

### Recherche et filtrage

Dans l'UI MLflow, filtrez par tags :
```
tags.phase = "baseline" AND metrics.roc_auc_mean > 0.75
```

## Interface MLflow

Pour visualiser vos expériences MLflow, lancez l'interface web depuis votre environnement uv:

**Option 1 - Depuis le terminal VSCode:**
```bash
cd /home/zmxw1768/Documents/oc_mlops
uv run mlflow ui
```

**Option 2 - Via l'environnement virtuel:**
```bash
source .venv/bin/activate
mlflow ui
```

Ensuite, ouvrez votre navigateur sur: **http://localhost:5000**

> 💡 Les données sont stockées localement dans le dossier `./mlruns` (paramètres, métriques et artifacts)

## 🔧 Modifier les tags d'un run existant

Si vous voulez ajouter/modifier des tags sur un run déjà terminé :

In [ ]:
# Exemple : Ajouter des tags à un run existant
import mlflow

# Méthode 1 : Par ID de run
run_id = "4b919d82f33d4f8c9b0066efbd98e3c7"  # Remplacez par votre run_id
client = mlflow.tracking.MlflowClient()

client.set_tag(run_id, "deployment_ready", "True")
client.set_tag(run_id, "validated_by", "Team Lead")
client.set_tag(run_id, "mlflow.note.content", "✅ Modèle validé pour production")

# Méthode 2 : Chercher et taguer plusieurs runs
experiment = mlflow.get_experiment_by_name("Default")  # Ou votre nom d'expérience
runs = mlflow.search_runs(
    experiment_ids=[experiment.experiment_id],
    filter_string="metrics.roc_auc_mean > 0.75"
)

for idx, run in runs.iterrows():
    client.set_tag(run.run_id, "high_performance", "True")
    print(f"✅ Tagged run {run.run_id[:8]}...")

In [ ]:
# Tags au niveau de l'EXPÉRIENCE (s'appliquent à tous les runs)
experiment_id = "148740929161326232"  # Votre experiment_id

client = mlflow.tracking.MlflowClient()
client.set_experiment_tag(experiment_id, "project", "Home Credit Default Risk")
client.set_experiment_tag(experiment_id, "dataset", "application_train.csv")
client.set_experiment_tag(experiment_id, "business_goal", "Minimize default cost")